In [1]:
# nsmc 텍스트 토큰나이징 한 pickle 데이터 불러오기
import pickle
with open('train_text.pk', 'rb') as f:
    train = pickle.load(f)

In [2]:
# import
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras


In [3]:
# 토크나이징, 패딩(제로패딩)
import numpy as np
# 문서만 추출
train_x = [doc for doc, _ in train]

# 토크나이징
tokenizer = Tokenizer(num_words=35000)
tokenizer.fit_on_texts(train_x)
# 학습된 모델 불러오기
model = keras.models.load_model("ncms_lstm")

In [4]:
# 후기 데이터 불러오기
import pymysql
conn = pymysql.connect(host='15.164.232.248', user='root', password='xx6677as', db='collection', charset='utf8mb4')
cur = conn.cursor()
cur.execute('select review from 29cm_lumir where product_no = 650636')
data = cur.fetchall()
cur.close()
conn.close()

In [5]:
data = list(data)

In [6]:
data[3][0]

'집을 이사하고 이것저것 새로운 가구들을 들였지만 느낌이 없어 조명으로 눈을 돌렸다가 만난 조명인데, 너무 이쁘네요 조명 들어오는 순간 거실에 감성충망하게 만들어주네요 거실이 넓은 편인데 밝기가 밝아 밤에는 이거 하나로도 지낼만하고요 유리라허 고민을 많이했는데 아래 지지도 꽤 무겁고 그 아래 고무가 식탁에 딱 붙어서 잘 떨어지지도 않을거 같네요 너무 마음에 듭니다!'

In [7]:
review = [row[0] for row in data]

In [8]:
review

['완벽하게 제가 찾던 조명이에요. 성능, 디자인, 가격까지  어떻게 하나도 빠지지않고 이렇게 완벽하게 마음에들수 있는지 ^^\n그중 타이머 기능은 자기전 항상 침대옆 조명을 켜놓는 사람들에겐 꼭 필요한 기능이죠. 조명을 켜놓고 자서 설잠을 자거나 조명을 끄려다 잠을 깨는일이 종종있어 정말 원하던 기능이였어요. \n심플하지만 볼륨감있는 디자인도 매력적이고 빛이 사방으로 퍼지지만 전혀 눈부시지 않고 책을 읽는데도 전혀 불편함 없고 어떤일을 하던 적당한 밝음에 밤에 메인등을 켤 일이 없을것 같아요. \n오랜만에 완벽한 구매였습니다.\n루미르조명 앞으로 계속 기대해봅니다 ^^',
 '방에 어울리는 조명을 고르기까지 많이 찾아보고 고민하다가 루미르 제품을 선택했는데 정말 만족스러워요. 처음 배송왔을 때 유리가 깨져서 와서 교환하는데 시간이 좀 걸렸지만 친절하게 응대해 주셨고 오늘 제대로 조립해서 켜봤어요. \n받침부분이 무게감이 있는 편이라 저렴해보이지 않고 흔들림없이 더 잘 지탱될 것 같아요. \n밝기 조절도 부드럽게 잘 되고 다루기 복잡하지 않고 심플해서 좋아요. 조명 하나 들어온 것 뿐인데 방 분위기가 달라져서 방에 들어갈 때마다 자꾸 눈이 가네요. 조명의 힘이 크다는 걸 다시 느꼈어요. 무엇보다 세 가지가 색 모두 최대 밝기로 켜도 눈이 안아픈 게 너무 마음에 들어요! 할인까지 해서 저렴한 가격에 이 정도 퀄리티의 조명을 구입해서 너무 만족합니다:)',
 '예전부터 루미르의 다른 조명 제품을 사용하고 있었어요. 눈에도 부담스럽지 않은 편안한 빛이고 군더더기 없는 디자인도 제 취향이라 만족하며 사용하고 있었는데, 신제품이 나왔다는 소식을 알게되어 바로 또 구매했습니다🙂 \n\n역시나 실망시키지 않네요. 심플하면서도 고급스럽고, 사진에는 차이가 잘 나오지 않지만 3가지로 색을 바꿀 수 있어서 다른 무드가 만들어지는 점도 좋아요. 밝기는 거실 테이블 위에 두었더니 스탠드만 켜두고 지내도 충분할 정도입니다. 세상을 밝힌다는 브랜드의 의미도 와닿았고,  매번 돈이 아깝

In [9]:
def preprocessing(txt):
    import re
    txt = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', txt)
    sw = set() # 집합자료형 선언
    with open('stopwords-ko.txt', encoding='utf-8') as f:
        for w in f:
            sw.add(w.replace('\n',''))
    doc = []
    from eunjeon import Mecab
    mecab = Mecab()
    for word in mecab.morphs(txt):
        if word not in sw and len(word) > 1:
            doc.append(word)
    return doc

In [10]:
# 학습한 머신을 이용한 데이터 predict 후 리스트에 담기
def predict_func(review,pre_data):
    for text in review:
        text = preprocessing(text)
        text = tokenizer.texts_to_sequences([text])
        text = pad_sequences(text, maxlen=50, padding='pre')
        pre_data.append((model.predict(text)))
    return pre_data

In [11]:
pre_data = []
out_data = predict_func(review,pre_data)

In [12]:
# 예측한 결과 값에 *100 후 소수 첫번째에서 반올림 후 score에 저장  
result = [row[0][0] for row in out_data]
score =[]
for num in result:
    score.append(round(num*100))
score

[100,
 100,
 100,
 100,
 100,
 100,
 100,
 4,
 100,
 100,
 71,
 87,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 67,
 100,
 100,
 97,
 100,
 0,
 95,
 99,
 100,
 94,
 2,
 100,
 100,
 100,
 100,
 1,
 100,
 100,
 100,
 98,
 100,
 1,
 0,
 100,
 100,
 100,
 100,
 99,
 100,
 9,
 100,
 100,
 100,
 100,
 14,
 73,
 100,
 100,
 88,
 100,
 86,
 100,
 100,
 100,
 100,
 100,
 4,
 100,
 100,
 100,
 0,
 100,
 100,
 92,
 100,
 0,
 100,
 97,
 1,
 100,
 12,
 5,
 0,
 100,
 100,
 100,
 2,
 97,
 100,
 100,
 100]

In [13]:
# 점수와 후기 데이터 하나의 튜플로 합치기
data = tuple(zip(score,review))

In [14]:
data

((100,
  '완벽하게 제가 찾던 조명이에요. 성능, 디자인, 가격까지  어떻게 하나도 빠지지않고 이렇게 완벽하게 마음에들수 있는지 ^^\n그중 타이머 기능은 자기전 항상 침대옆 조명을 켜놓는 사람들에겐 꼭 필요한 기능이죠. 조명을 켜놓고 자서 설잠을 자거나 조명을 끄려다 잠을 깨는일이 종종있어 정말 원하던 기능이였어요. \n심플하지만 볼륨감있는 디자인도 매력적이고 빛이 사방으로 퍼지지만 전혀 눈부시지 않고 책을 읽는데도 전혀 불편함 없고 어떤일을 하던 적당한 밝음에 밤에 메인등을 켤 일이 없을것 같아요. \n오랜만에 완벽한 구매였습니다.\n루미르조명 앞으로 계속 기대해봅니다 ^^'),
 (100,
  '방에 어울리는 조명을 고르기까지 많이 찾아보고 고민하다가 루미르 제품을 선택했는데 정말 만족스러워요. 처음 배송왔을 때 유리가 깨져서 와서 교환하는데 시간이 좀 걸렸지만 친절하게 응대해 주셨고 오늘 제대로 조립해서 켜봤어요. \n받침부분이 무게감이 있는 편이라 저렴해보이지 않고 흔들림없이 더 잘 지탱될 것 같아요. \n밝기 조절도 부드럽게 잘 되고 다루기 복잡하지 않고 심플해서 좋아요. 조명 하나 들어온 것 뿐인데 방 분위기가 달라져서 방에 들어갈 때마다 자꾸 눈이 가네요. 조명의 힘이 크다는 걸 다시 느꼈어요. 무엇보다 세 가지가 색 모두 최대 밝기로 켜도 눈이 안아픈 게 너무 마음에 들어요! 할인까지 해서 저렴한 가격에 이 정도 퀄리티의 조명을 구입해서 너무 만족합니다:)'),
 (100,
  '예전부터 루미르의 다른 조명 제품을 사용하고 있었어요. 눈에도 부담스럽지 않은 편안한 빛이고 군더더기 없는 디자인도 제 취향이라 만족하며 사용하고 있었는데, 신제품이 나왔다는 소식을 알게되어 바로 또 구매했습니다🙂 \n\n역시나 실망시키지 않네요. 심플하면서도 고급스럽고, 사진에는 차이가 잘 나오지 않지만 3가지로 색을 바꿀 수 있어서 다른 무드가 만들어지는 점도 좋아요. 밝기는 거실 테이블 위에 두었더니 스탠드만 켜두고 지내도 충분할 정도입니다. 세상을 밝힌다

In [15]:
#점수 데이터 데이터 베이스에 업데이트
#conn = pymysql.connect(host='15.164.232.248', user='root', password='xx6677as', db='collection', charset='utf8mb4')
# Connection 으로부터 Cursor 생성
#cur = conn.cursor()
#for d in data:
    #cur.execute("update 29cm_lumir set score = %s where review = %s", d)
#conn.commit()
#conn.close()